In [2]:
from robot_vlp.config import INTERIM_DATA_DIR, PROCESSED_DATA_DIR, FIGURES_DIR, MODELS_DIR, EXPERIMENT_DATA_DIR
import pickle
import numpy as np
import keras
import robot_vlp.data.preprocessing as p
import matplotlib.pyplot as plt
import robot_vlp.data_collection.communication as c
import pandas as pd
from kerastuner import HyperParameters
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, BatchNormalization, LayerNormalization, Input, Bidirectional, Attention, Add
from tensorflow.keras.models import Model
import kerastuner as kt
import json

import robot_vlp.modeling.gen_cnc_vlp_model as vlp

import robot_vlp.data_collection.experment_processing as ep

# import robot_vlp.data.odometer_path_navigation as pg
# import robot_vlp.plots.model_performance_plotting as pp
import robot_vlp.modeling.rnn as rnn
import robot_vlp.stats.navigation_performance as nav

from tensorflow import keras
import tensorflow as tf

%load_ext autoreload
%autoreload 2

import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')


from robot_vlp.modeling.rnn_config import GLOBAL_CONFIG

/var/folders/j0/3mr_0p157c9d4qgnmrzwc8nm0000gn/T/ipykernel_7689/2091869221.py:9: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperParameters


In [3]:
vlp_models = vlp.load_vlp_models()
vlp_model = vlp_models['high_acc']
# df_lst = []
# for i in range(10):
#     path = EXPERIMENT_DATA_DIR/f'Robot/exp1_{i}.csv'
#     df_lst.append(ep.process_robot_exp_file(path, vlp_model))

df_lst = []
for i in range(10):
    test_file = INTERIM_DATA_DIR / 'exp_vive_navigated_paths'/f'exp1_{i}_high_acc.csv'
    df = pd.read_csv(test_file)
    df_lst.append(df)

train_files = df_lst[:-2]
valid_files = df_lst[-2:-1]
test_files = df_lst[-1:]

In [4]:
import robot_vlp.modeling.rnn as rnn


def expand(y):
    y_rad = y[:,2] 
    y_angles = np.column_stack((np.sin(y_rad), np.cos(y_rad)))
    return [y[:,:2], y_angles]

# Define a cosine similarity–based loss function for headings
def cosine_loss(y_true, y_pred):
    # Use tf.keras.losses.CosineSimilarity which returns negative values (max similarity = -1)
    cos_sim = tf.keras.losses.CosineSimilarity(axis=1)(y_true, y_pred)
    # Convert to a loss (0 when identical, higher when misaligned)
    return 1 + cos_sim  # When vectors are identical, cos_sim = -1, so loss becomes 0


def preprocess_df(df):
    X = df[['vlp_x_hist', 'vlp_y_hist','vlp_heading_hist_rad','vlp_heading_change_rad', 'encoder_heading_change_rad', 'encoder_heading_hist_rad', 'encoder_x_hist','encoder_y_hist']].values
    y = df[['x_hist', 'y_hist','heading_hist_rad']].values
    X_win, y_win, m_win = p.window_data(X, y, y, overlap = 0.999999, window_len = 25)
    return X_win, y_win, m_win


def read_csv_to_train(file_list):
    X_lst = []
    y_lst = []
    m_lst = []

    for df in file_list:
        X_win, y_win, m_win = preprocess_df(df)
        X_lst.append(X_win)
        y_lst.append(y_win)
        m_lst.append(m_win)


    X = np.concatenate(X_lst, axis = 0)
    y = np.concatenate(y_lst, axis = 0)
    m = np.concatenate(m_lst, axis = 0)

    return X, y, m

X_train, y_train, _ = read_csv_to_train(train_files)
X_valid, y_valid, _ = read_csv_to_train(valid_files)
X_test, y_test, _ = read_csv_to_train(test_files)

X_train = np.nan_to_num(X_train, nan = 0)
X_valid = np.nan_to_num(X_valid, nan = 0)
X_test = np.nan_to_num(X_test, nan = 0)

# after you've built `X_train` with shape (N, window, 8):
flat = X_train.reshape(-1, X_train.shape[-1])
rnn.global_norm.adapt(flat)

# 2) grab its statistics and push them into your module globals
rnn.FEAT_MEAN = rnn.global_norm.mean.numpy()
rnn.FEAT_VAR  = rnn.global_norm.variance.numpy()



## RNN Hyperparameter tuning

### Stage 1 parameter tuning - neurons and layers

In [5]:

tuner = kt.RandomSearch(
    rnn.build_architecture_model,   # ← use your config‑driven builder
    objective="val_loss",
    max_trials=200,
    directory="rnn_tuning",
    project_name="rnn_random_search",
    overwrite=False
)

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    "logs/random_search", histogram_freq=1
)

tuner.search(
    X_train,
    expand(y_train),
    epochs=100,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=5, restore_best_weights=True
        ),
        tensorboard_callback
    ]
)


Reloading Tuner from rnn_tuning/rnn_random_search/tuner0.json


In [6]:
best_model = tuner.get_best_models(num_models=1)[0]

/Users/tyrelglass/miniforge3/envs/robot-vlp/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [7]:
# Extract the top 10 trials from your tuner
best_trials = tuner.oracle.get_best_trials(num_trials=10)

rows = []
for trial in best_trials:
    hp = trial.hyperparameters
    # Pull out architecture hyperparameters
    num_layers = hp.get('num_layers')
    rec_units = [hp.get(f'recurrent_units_{i}') for i in range(num_layers)]
    # Pad to a fixed width (max 4 layers)
    rec_units += [None] * (4 - len(rec_units))
    # Get the best validation loss achieved by this trial
    val_loss = trial.metrics.get_best_value('val_loss')
    
    rows.append({
        'trial_id': trial.trial_id,
        'val_loss': val_loss,
        'num_layers': num_layers,
        'units_layer_0': rec_units[0],
        'units_layer_1': rec_units[1],
        'units_layer_2': rec_units[2],
        'units_layer_3': rec_units[3],
    })

# Build and display a DataFrame
df_top10 = pd.DataFrame(rows).sort_values('val_loss').reset_index(drop=True)
df_top10



# 1) Grab the best HP from stage 1
best_hp = tuner.get_best_hyperparameters(1)[0]

# 2) Update GLOBAL_CONFIG in memory
GLOBAL_CONFIG["best_architecture"] = {
    "num_layers": best_hp.get("num_layers"),
    "recurrent_units": [
        best_hp.get(f"recurrent_units_{i}") 
        for i in range(best_hp.get("num_layers"))
    ]
}

# 3) (Optional) Persist to disk as config.json
with open("config.json", "w") as f:
    json.dump(GLOBAL_CONFIG, f, indent=2)

print("✅ GLOBAL_CONFIG updated:", GLOBAL_CONFIG["best_architecture"])

✅ GLOBAL_CONFIG updated: {'num_layers': 2, 'recurrent_units': [32, 8]}


In [8]:
df_top10

,trial_id,val_loss,num_layers,units_layer_0,units_layer_1,units_layer_2,units_layer_3
0,119,0.004202,2,32,8.0,NaN,NaN
1,158,0.004238,1,32,NaN,NaN,NaN
2,157,0.004388,2,64,16.0,NaN,NaN
3,149,0.004633,2,64,16.0,NaN,NaN
4,052,0.004742,2,16,8.0,NaN,NaN
5,125,0.004798,2,64,64.0,NaN,NaN
6,079,0.004833,2,128,128.0,NaN,NaN
7,115,0.004844,4,64,8.0,32.0,16.0
8,055,0.005066,2,128,64.0,NaN,NaN
9,040,0.005203,3,64,64.0,32.0,NaN


### Stage 2 tuning - regularization

In [9]:
tuner = kt.RandomSearch(
    rnn.build_regularization_model,  # Your stage 2 model-building function
    objective="val_loss",
    max_trials=200,
    directory="rnn_tuning_stage2",  # New directory for stage 2
    project_name="rnn_reg_tuning",
    overwrite=False
)

# ✅ Enable TensorBoard Logging
tensorboard_callback = tf.keras.callbacks.TensorBoard("logs/random_search_2", histogram_freq=1)

# ✅ Start tuning with random search
tuner.search(
    X_train, expand(y_train),
    epochs = 50,
    validation_split=0.2,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
        tensorboard_callback
    ]
)

Reloading Tuner from rnn_tuning_stage2/rnn_reg_tuning/tuner0.json


In [10]:
# get the top 10 trials
best_trials = tuner.oracle.get_best_trials(num_trials=100)

rows = []
for trial in best_trials:
    hp = trial.hyperparameters
    num_layers = GLOBAL_CONFIG["best_architecture"]["num_layers"]
    # pull out each layer’s regularization HPs
    dr   = [ hp.get(f"dropout_{i}")           for i in range(num_layers) ]
    rdr  = [ hp.get(f"recurrent_dropout_{i}") for i in range(num_layers) ]
    bn   = [ hp.get(f"batch_norm_{i}")        for i in range(num_layers) ]
    ln   = [ hp.get(f"layer_norm_{i}")        for i in range(num_layers) ]
    val_loss = trial.metrics.get_best_value("val_loss")

    rows.append({
      "trial_id":       trial.trial_id,
      "val_loss":       val_loss,
      **{f"dropout_{i}":           dr[i]  for i in range(num_layers)},
      **{f"recurrent_dropout_{i}": rdr[i] for i in range(num_layers)},
      **{f"batch_norm_{i}":        bn[i]  for i in range(num_layers)},
      **{f"layer_norm_{i}":        ln[i]  for i in range(num_layers)},
    })

df_reg_top100 = pd.DataFrame(rows).sort_values("val_loss").reset_index(drop=True)
df_reg_top100


,trial_id,val_loss,dropout_0,dropout_1,recurrent_dropout_0,recurrent_dropout_1,batch_norm_0,batch_norm_1,layer_norm_0,layer_norm_1
0,064,0.006153,0.000,0.010,0.025,0.050,False,False,True,False
1,072,0.006451,0.010,0.000,0.010,0.000,False,False,True,True
2,029,0.006678,0.010,0.050,0.010,0.050,False,False,True,False
3,088,0.006813,0.050,0.000,0.010,0.010,False,False,False,False
4,192,0.006844,0.010,0.000,0.025,0.050,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...
95,014,0.010407,0.025,0.025,0.050,0.050,False,True,True,True
96,074,0.010456,0.010,0.050,0.000,0.010,False,True,True,True
97,094,0.010457,0.050,0.025,0.025,0.000,False,False,False,True
98,068,0.010463,0.010,0.000,0.000,0.050,True,False,True,True


### Check no regularisation

In [19]:
from keras_tuner.engine.hyperparameters import HyperParameters
class FixedHyperParameters(HyperParameters):
    def __init__(self):
        super().__init__()

    def Choice(self, name, values, default=None):
        return 0.0  # Always return 0.0 for dropout and recurrent_dropout

    def Boolean(self, name):
        return False  # Always return False for all normalization options

fixed_hp = FixedHyperParameters()
baseline_model = rnn.build_regularization_model(fixed_hp)

history = baseline_model.fit(
    X_train, expand(y_train),
    validation_split=0.2,
    epochs=50,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),

    ]
)


Epoch 1/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - angle_output_loss: 0.4510 - loc_output_loss: 0.0613 - loss: 0.5124 - val_angle_output_loss: 0.2920 - val_loc_output_loss: 0.0084 - val_loss: 0.3003
Epoch 2/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - angle_output_loss: 0.2309 - loc_output_loss: 0.0070 - loss: 0.2379 - val_angle_output_loss: 0.1049 - val_loc_output_loss: 0.0053 - val_loss: 0.1099
Epoch 3/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - angle_output_loss: 0.0890 - loc_output_loss: 0.0038 - loss: 0.0928 - val_angle_output_loss: 0.0537 - val_loc_output_loss: 0.0030 - val_loss: 0.0564
Epoch 4/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - angle_output_loss: 0.0467 - loc_output_loss: 0.0023 - loss: 0.0490 - val_angle_output_loss: 0.0320 - val_loc_output_loss: 0.0022 - val_loss: 0.0340
Epoch 5/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - angle_output_loss: 0.0316 - loc_output_loss: 0.0017 - loss: 0.0333 - val_angle_output_loss: 0.0250 - val_loc_output_loss: 0.0018 - val_los

In [13]:
df_reg_top100

,trial_id,val_loss,dropout_0,dropout_1,recurrent_dropout_0,recurrent_dropout_1,batch_norm_0,batch_norm_1,layer_norm_0,layer_norm_1
0,064,0.006153,0.000,0.010,0.025,0.050,False,False,True,False
1,072,0.006451,0.010,0.000,0.010,0.000,False,False,True,True
2,029,0.006678,0.010,0.050,0.010,0.050,False,False,True,False
3,088,0.006813,0.050,0.000,0.010,0.010,False,False,False,False
4,192,0.006844,0.010,0.000,0.025,0.050,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...
95,014,0.010407,0.025,0.025,0.050,0.050,False,True,True,True
96,074,0.010456,0.010,0.050,0.000,0.010,False,True,True,True
97,094,0.010457,0.050,0.025,0.025,0.000,False,False,False,True
98,068,0.010463,0.010,0.000,0.000,0.050,True,False,True,True


In [21]:
# Run this right after tuner.search(...)
best_hp = tuner.get_best_hyperparameters(1)[0]

# Build your new regularization_defaults entry
num_layers = GLOBAL_CONFIG["best_architecture"]["num_layers"]
GLOBAL_CONFIG["regularization_defaults"] = {
  "dropout": [best_hp.get(f"dropout_{i}") for i in range(num_layers)],
  "recurrent_dropout": [best_hp.get(f"recurrent_dropout_{i}") for i in range(num_layers)],
  "batch_norm": [best_hp.get(f"batch_norm_{i}") for i in range(num_layers)],
  "layer_norm": [ best_hp.get(f"layer_norm_{i}") for i in range(num_layers) ]
}
# 3) (Optional) Persist to disk as config.json
with open("config.json", "w") as f:
    json.dump(GLOBAL_CONFIG, f, indent=2)
print("✅ GLOBAL_CONFIG updated:", GLOBAL_CONFIG["best_architecture"])

✅ GLOBAL_CONFIG updated: {'num_layers': 2, 'recurrent_units': [32, 8]}


### Stage 3 tuning - Optimization parameters

In [26]:
tuner = kt.RandomSearch(
    rnn.build_optimization_model,  # Stage 3 model function
    objective="val_loss",
    max_trials=200,
    directory="rnn_tuning_stage3",
    project_name="rnn_opt_tuning",
    overwrite=False
)

tensorboard_callback = tf.keras.callbacks.TensorBoard("logs/opt_tuning", histogram_freq=1)

tuner.search(
    X_train, expand(y_train),
    epochs=100,
    validation_split=0.2,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
        tensorboard_callback
    ]
)


Reloading Tuner from rnn_tuning_stage3/rnn_opt_tuning/tuner0.json


In [27]:

best_hp = tuner.get_best_hyperparameters(1)[0]

opt_cfg = {
    "lr":         float(best_hp.get("lr")),
    "scheduler":  best_hp.get("scheduler"),
    "optimizer":  best_hp.get("optimizer"),
}

if opt_cfg["scheduler"] == "cosine":
    opt_cfg["decay_steps"] = int(best_hp.get("decay_steps_cosine"))
elif opt_cfg["scheduler"] == "exponential":
    # note the full name here:
    opt_cfg["decay_steps"] = int(best_hp.get("decay_steps_exponential"))
    opt_cfg["decay_rate"]  = float(best_hp.get("decay_rate_exp"))

GLOBAL_CONFIG["optimization_defaults"] = opt_cfg
with open("config.json", "w") as f:
    json.dump(GLOBAL_CONFIG, f, indent=2)

print("✅ NEW optimization_defaults:", GLOBAL_CONFIG["optimization_defaults"])


✅ NEW optimization_defaults: {'lr': 0.0033727437553468355, 'scheduler': 'none', 'optimizer': 'nadam'}


In [28]:
# 1) Grab the top-10 (or fewer if you only ran 5) trials
best_trials = tuner.oracle.get_best_trials(num_trials=10)

rows = []
for trial in best_trials:
    hp       = trial.hyperparameters
    val_loss = trial.metrics.get_best_value("val_loss")

    # these are the tuned HPs in build_optimization_model:
    lr          = hp.get("lr")
    sched       = hp.get("scheduler")
    # conditionally pull out the scheduler params
    decay_cos   = hp.get("decay_steps_cosine") if sched == "cosine"      else None
    decay_exp   = hp.get("decay_steps_exponential")   if sched == "exponential" else None
    rate_exp    = hp.get("decay_rate_exp")    if sched == "exponential" else None
    optimizer   = hp.get("optimizer")

    rows.append({
        "trial_id":            trial.trial_id,
        "val_loss":            val_loss,
        "lr":                  lr,
        "scheduler":           sched,
        "decay_steps_cosine":  decay_cos,
        "decay_steps_exponential": decay_exp,
        "decay_rate_exponential":  rate_exp,
        "optimizer":           optimizer,
    })

# 2) Build & sort your DataFrame
df_opt_top10 = (
    pd.DataFrame(rows)
      .sort_values("val_loss")
      .reset_index(drop=True)
)

# 3) Inspect
print(df_opt_top10)


  trial_id  val_loss        lr    scheduler  decay_steps_cosine  \
0      101  0.003747  0.003373         none                 NaN   
1      179  0.004268  0.003407  exponential                 NaN   
2      090  0.004286  0.009201       cosine             10846.0   
3      015  0.004437  0.006924       cosine              9724.0   
4      167  0.004810  0.001150         none                 NaN   
5      152  0.004883  0.001608         none                 NaN   
6      174  0.004968  0.001852         none                 NaN   
7      166  0.004982  0.005711         none                 NaN   
8      180  0.005052  0.005048         none                 NaN   
9      018  0.005248  0.001860  exponential                 NaN   

   decay_steps_exponential  decay_rate_exponential optimizer  
0                      NaN                     NaN     nadam  
1                  13464.0                    0.82      adam  
2                      NaN                     NaN     nadam  
3         

### Stage 4 Tuning - Other hyperparameters

In [46]:
tuner = kt.GridSearch(
    rnn.build_stage4_model,  # Stage 4 model function
    objective=kt.Objective("val_loss", direction="min"),
    directory="rnn_tuning_stage4",  # New directory for stage 4
    project_name="rnn_extra_tuning",
    overwrite=False
)

tensorboard_callback = tf.keras.callbacks.TensorBoard("logs/stage4", histogram_freq=1)

tuner.search(
    X_train, expand(y_train),
    epochs=100,
    validation_split=0.2,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
        tensorboard_callback
    ]
)


Reloading Tuner from rnn_tuning_stage4/rnn_extra_tuning/tuner0.json


In [47]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:")
print(best_hps.values)

Best hyperparameters:
{'sequence_length': 20}


In [48]:

# 1) After your tuner.search(...)
best_hp = tuner.get_best_hyperparameters(1)[0]

# 2) Pull out the winning window‐length choice
best_seq = int(best_hp.get("sequence_length"))

# 3) Update your in‐memory config
GLOBAL_CONFIG["sequence_length"]["use_length"] = best_seq

# 4) Persist it to disk
with open("config.json", "w") as f:
    json.dump(GLOBAL_CONFIG, f, indent=2)

print(f"✅ GLOBAL_CONFIG.sequence_length.use_length set to {best_seq}")


✅ GLOBAL_CONFIG.sequence_length.use_length set to 20


In [49]:
# collect all trials
rows = []
for trial in tuner.oracle.trials.values():
    hp       = trial.hyperparameters
    seq_len  = hp.get("sequence_length")
    # get the best val_loss this trial ever saw
    val_loss = trial.metrics.get_best_value("val_loss")
    rows.append({
        "trial_id":        trial.trial_id,
        "sequence_length": seq_len,
        "val_loss":        val_loss
    })

# build a DataFrame and sort by loss
df_stage4 = (
    pd.DataFrame(rows)
      .sort_values("val_loss")
      .reset_index(drop=True)
)

df_stage4


,trial_id,sequence_length,val_loss
0,0003,20,0.003779
1,0002,15,0.004495
2,0001,10,0.005148
3,0000,5,0.006369
4,0004,25,0.007162


### Final model

In [51]:
best_model = rnn.build_final_model()


history = best_model.fit(
    x = X_train[:,-20:,:], 
    y = expand(y_train),
    validation_data=(X_valid[:,-20:,:], expand(y_valid)),
    epochs=500, 
    batch_size=32, 
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)]
)

Epoch 1/500
374/374 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - angle_output_loss: 0.2993 - loc_output_loss: 0.0517 - loss: 0.3511 - val_angle_output_loss: 0.0357 - val_loc_output_loss: 0.0037 - val_loss: 0.0392
Epoch 2/500
374/374 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - angle_output_loss: 0.0381 - loc_output_loss: 0.0037 - loss: 0.0417 - val_angle_output_loss: 0.0144 - val_loc_output_loss: 0.0017 - val_loss: 0.0151
Epoch 3/500
374/374 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - angle_output_loss: 0.0216 - loc_output_loss: 0.0021 - loss: 0.0237 - val_angle_output_loss: 0.0119 - val_loc_output_loss: 0.0013 - val_loss: 0.0125
Epoch 4/500
374/374 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - angle_output_loss: 0.0184 - loc_output_loss: 0.0015 - loss: 0.0200 - val_angle_output_loss: 0.0118 - val_loc_output_loss: 0.0012 - val_loss: 0.0122
Epoch 5/500
374/374 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - angle_output_loss: 0.0157 - loc_output_loss: 0.0013 - loss: 0.0170 - val_angle_output_loss: 0.0082 - val_loc_output_loss: 0.0011 - va

In [52]:
best_model.save(MODELS_DIR / 'navigation_neural_nets/rnn.keras')

In [53]:
best_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 20, 8)     │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feature_norm        │ (None, 20, 8)     │         17 │ input[0][0]       │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_0 (LSTM)       │ (None, 20, 32)    │      5,248 │ feature_norm[232… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_norm_0        │ (None, 20, 32)    │         64 │ lstm_0[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 8)         │      1,312 │ layer_norm_0[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ loc_output (Dense)  │ (None, 2)         │         18 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ angle_output        │ (None, 2)         │         18 │ lstm_1[0][0]      │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 20,000 (78.13 KB)

 Trainable params: 6,660 (26.02 KB)

 Non-trainable params: 17 (72.00 B)

 Optimizer params: 13,323 (52.05 KB)

# MLP implementation

In [ ]:
import robot_vlp.modeling.mlp as mlp

In [ ]:
X_test_scaled.shape

In [ ]:
model = mlp.build_default_mlp()
model.summary()





history = model.fit(
    x = X_train_scaled[:,-1,:], 
    y = expand(y_train),
    validation_data = (X_valid_scaled[:,-1,:],  expand(y_valid)),
    epochs=300, 
    batch_size=32, 
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)] )


pre_loc, pre_ang = model.predict(X_test_scaled[:,-1,:])
MLP_loc_errs = calc_loc_err(pre_loc, y_test[:,:2])

MLP_ang_errs = vec_to_ang(pre_ang) - y_test[:,2]
MLP_ang_errs = np.array([normalize_angle_deg(ang) for ang in MLP_ang_errs])


vlp_ang_errs = y_test[:,2] -  X_test[:,-1,2]
vlp_ang_errs = np.array([normalize_angle_deg(ang) for ang in vlp_ang_errs])


vlp_loc_errs = calc_loc_err(X_test[:,-1,:2], y_test[:,:2])

In [ ]:
print(f"vlp pos errs:{vlp_loc_errs.mean()}")
print(f"MLP pos errs:{MLP_loc_errs.mean()}")


print(f"VLP heading errs:{np.abs(vlp_ang_errs).mean()}")
print(f"MLP heading errs:{np.abs(MLP_ang_errs).mean()}")